## Spark Basics

Spark requires communication and synchronization between a number of computers connected trough an ethernet connection. The data structure that manages this communication is a **SparkContext**. A program needs only one SparkContext to run. In a stand-alone pyspark script the SparkContext Object has to be initialized explicitly. 

When running pyspark in a notebook the notebook manager initializes a SparkContext named **sc**

In [1]:
sc

### RDDs
RDDs (or Resilient Distributed DataSets) are the fundamental data structure used in Spark. You can consider it to be an array whose elements are stored on several computers. 

The simplest way of creating an RDD is by initializing it from a regular array using the method `sc.parallelize`

(In this notebook we will use very tiny RDDs, to help understanding. The real utility of RDDs is for lists with millions or billions of items, which do not fit in the memory of a single computer)

In [2]:
RDD=sc.parallelize([0,1,2])
RDD

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:423

An RDD can be converted back to a regular list, residing on the head node using the method `.collect`

In [3]:
RDD.collect()

[0, 1, 2]

### Map
The map method applies a given operation to each element in the RDD, thus creating a new RDD.

In [4]:
RDD.map(lambda x: x*x).collect()

[0, 1, 4]

### Reduce
The reduce operation takes as input an RDD and repeatedly applies a 2-to-1 operation such as summation or max.
A 2-to-1 operation is an operation that takes as input two input items of some type and outputs a single item of the same type.

The simplest example of a 2-to-1 operation is the sum:

In [5]:
RDD.reduce(lambda x,y: x+y)

3

Here is an example of a reduce operation that finds the shortest string in an RDD of strings.

In [6]:
words=['this','is','the','best','mac','ever']
wordRDD=sc.parallelize(words)
wordRDD.reduce(lambda w,v: w if len(w)<len(v) else v)

'is'

Lambda functions can produce compact code. However, you can use regular functions instead. This is a better choice when the operation is too complex to fit in a single line.

For example suppose that we want to find the last word in a lexicographical order among the longest words in the list.
We could achieve that as follows

In [7]:
def largerThan(x,y):
    if len(x)>len(y): return x
    elif len(y)>len(x): return y
    else:  #lengths are equal, compare lexicographically
        if x>y: 
            return x
        else: 
            return y
        
wordRDD.reduce(largerThan)

'this'

### Reduce operations **must not depend on the order of application**

You can think about the reduce operation as a binary tree where the leaves are the elements of the list and the root is the final result. Each triplet of the form (parent, child1, child2) corresponds to a single application of the reduce function. There are many ways of arranging this binary tree. **all of these ways have to yield the same final result**. In addition, the order of the elements in the list must not change the result. In particular, reversing the order of the operands in a reduce function must not change the outcome. 

For example the arithmetic operations multiply `*` and add `+` can be used in a reduce, but the operations subtract `-` and divide `/` cannot.

Doing so will not raise an error, but the result is unpredictable.

In [8]:
print RDD.collect()
RDD.reduce(lambda x,y: x-y)

[0, 1, 2]


1

### Combining operations

The method `map` takes as input an RDD and returns an RDD. Similarly the method `reduce` takes as input an RDD and returns a single element. We can therefor cascade map and reduce operations to perform more complex operations in one line.

Suppose we want to compute the sum of the squares of the elements in the RDD. We can either write it in the traditional way:

In [9]:
#separate commands
Squares=RDD.map(lambda x:x*x)
Squares.reduce(lambda x,y:x+y)

5

Or we can combine them into a single cascaded command

In [10]:
#cascaded commands
RDD.map(lambda x:x*x)\
   .reduce(lambda x,y:x+y)

5

These two expressions are equivalent, and we might expect that the more basic one is the first, where the commands 
are separate, and that the python compiler translates the cascaded commands into machine code that corresponds to the separate commands.

It turns out that the opposite is true, it is the cascaded form that is closer to the machine code, and spark identifies cascading operations even when they are expressed in a non-cascaded way.

The explanation of this surprising behaviour is related to the notion of lazy evaluation in scala and is explained in [spark programming guide/RDD operations](http://spark.apache.org/docs/latest/programming-guide.html#rdd-operations)

### An instructive mistake
Here is another way to compute the sum of the squares using a single reduce command. What is wrong with it?

In [11]:
RDD.reduce(lambda x,y: x*x+y*y)


25

### getting some information about an RDD
RDD's typically have hundreds of thousands of elements. It usually makes no sense to print out the content of a whole RDD. Here are some ways to get manageable amounts of information about an RDD

In [12]:
n=10000;
B=sc.parallelize(range(n))

In [13]:
#find the number of elements in the RDD
B.count()

10000

In [14]:
# get the first few elements of an RDD
print 'first element=',B.first()
print 'first 5 elements = ',B.take(5)

first element= 0
first 5 elements =  [0, 1, 2, 3, 4]


#### Sampling an RDD
The method `RDD.sample(withReplacement,p)` generates a sample of the elements of the RDD. where
- `withReplacement` is a boolean flag indicating whether or not a an element in the RDD can be sampled more than once.
- `p` is the probability of accepting each element into the sample. Note that as the sampling is performed independently in each partition, the number of elements in the sample changes from sample to sample.

In [15]:
# get a sample whose expected size is m
m=5.
B.sample(False,m/n).collect()

[999, 1483, 7027]

#### filtering an RDD
The method `RDD.filter(func)` Return a new dataset formed by selecting those elements of the source on which func returns true.


In [16]:
# How many positive numbers?
B.filter(lambda n: n > 0).count()

9999

#### Remove duplicate elements in RDD
The method `RDD.distinct(numPartitions=None)` Returns a new dataset that contains the distinct elements of the source dataset 

* The number of partitions is specified through the **numPartitions** argument. Each of this partitions is potentially on different machine.


In [17]:
# Remove duplicate element in DuplicateRDD, we get distinct RDD
DuplicateRDD = sc.parallelize([1,1,2,2,3,3])
DistinctRDD = DuplicateRDD.distinct()
DistinctRDD.collect()


[2, 1, 3]

#### flatmap an RDD
The method `RDD.flatMap(func)` is similar to map, but each input item can be mapped to 0 or more output items (so func should return a Seq rather than a single item).

In [18]:
text=["you are my sunshine","my only sunshine"]
text_file = sc.parallelize(text)
# map each line in text to a list of words
print 'map:',text_file.map(lambda line: line.split(" ")).collect()
# create a single list of words by combining the words from all of the lines
print 'flatmap:',text_file.flatMap(lambda line: line.split(" ")).collect()

map: [['you', 'are', 'my', 'sunshine'], ['my', 'only', 'sunshine']]
flatmap: ['you', 'are', 'my', 'sunshine', 'my', 'only', 'sunshine']


### Set operations
In this part, we explore set operations including **union**,**intersection**,**subtract**, **cartesian** in pyspark
1. union(other)
 * Return the union of this RDD and another one.
2. intersection(other)
 * Return the intersection of this RDD and another one. The output will not contain any duplicate elements, even if the input RDDs did.Note that this method performs a shuffle internally.
3. subtract(other, numPartitions=None)
 * Return each value in self that is not contained in other.
4. cartesian(other)
 * Return the Cartesian product of this RDD and another one, that is, the RDD of all pairs of elements (a, b) where **a** is in **self** and **b** is in **other**.


In [19]:
rdd1 = sc.parallelize([1, 1, 2, 3])
rdd2 = sc.parallelize([1, 3, 4, 5])
print rdd1.union(rdd2).collect()
print rdd1.intersection(rdd2).collect()
print rdd1.subtract(rdd2).collect()
print rdd1.cartesian(rdd2).collect()

[1, 1, 2, 3, 1, 3, 4, 5]
[1, 3]
[2]
[(1, 1), (1, 3), (1, 1), (1, 3), (1, 4), (1, 5), (1, 4), (1, 5), (2, 1), (2, 3), (3, 1), (3, 3), (2, 4), (2, 5), (3, 4), (3, 5)]


### HW1
The goal of this first HW is to find the most common combinations of consecutive words in the text of moby dick.

Unigrams, bigrams, and in general $k$-grams 1,2 or k words that appear consecutively in a single sentence.
Consider the sentence  
> to know you is to love you.

This sentence containst:  

> **Unigrams (single words):** to(2 times), know(1), you(2), is(1), love(1)  
> **Bigrams:** "to know","know you","you is", "is to","to love", "love you"  (all 1)  
> **Trigrams:** "to know you", "know you is", ...  

Your task is to:
1. Convert all text to lower case, remove all punctuations other than periods.
1. Split the text into sentences using periods.
2. Count the occurance of each word and of each 2,3,4,5 - gram
3. List the 10 most common elements for each order (word, bigram, trigram...). For each element list the sequence of words and the number of occurances.

This text is easily short enough to process using a single core using standard python. However, you are required to solve it using RDD's for the whole process. At the very end you can use `.take(10)` to bring the results to the central node for pringing.

In [20]:
# Creating an RDD from a local file
textRDD=sc.textFile('../../data/Moby-Dick.txt')

In [21]:
!ls -l ../../data

total 2464
-rw-r--r--@ 1 yoavfreund  503    1257260 Jan  3  2013 Moby-Dick.txt
-rw-r--r--  1 yoavfreund  staff      679 Sep  8  2015 states.txt
